### MLP多层感知机

In [ ]:
from utils.dataset_utils import get_classes_indexes_counts
from sklearn.metrics import confusion_matrix
# 导入必要的库
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report, accuracy_score
import scipy.io as sio  # 从.mat文件中读取数据集

# 加载鸢尾花数据集
# data = load_iris()
mat_data = sio.loadmat('../data/dataset/Satellite.mat')
X = mat_data['X']  # 特征
y = mat_data['Y'][:, 0]  # 标签

# 将数据集分为训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=100)
classes, counts = get_classes_indexes_counts(y_test)
print(counts)
# 数据标准化
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# 构建并训练MLP模型
mlp = MLPClassifier(hidden_layer_sizes=(10, 20), max_iter=1000, random_state=42)
mlp.fit(X_train, y_train)
index_pred_proba = mlp.predict_proba(X_test)
# 预测和评估模型
y_pred = mlp.predict(X_test)

# 输出结果
print("准确率:", accuracy_score(y_test, y_pred))
print("\n分类报告:\n", classification_report(y_test, y_pred))
# 打印混淆矩阵
print("Confusion Matrix:")
cm = confusion_matrix(y_test, y_pred)
print(cm)
print(cm.sum(axis=1))
print(cm.diagonal())
print(cm.diagonal().sum())
print(index_pred_proba.shape)

In [ ]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import confusion_matrix
import numpy as np

# 加载鸢尾花数据集
iris = load_iris()
X = iris.data
y = iris.target
class_names = iris.target_names

# 数据拆分为训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

# 数据标准化
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# 定义并训练MLP模型
mlp = MLPClassifier(hidden_layer_sizes=(10, 10), max_iter=1000, random_state=42)
mlp.fit(X_train, y_train)

# 测试集预测
y_pred = mlp.predict(X_test)

# 混淆矩阵
cm = confusion_matrix(y_test, y_pred)

# 输出结果
print("类别信息:")
for i, class_name in enumerate(class_names):
    total_count = np.sum(y_test == i)
    correct_count = cm[i, i]
    incorrect_count = total_count - correct_count
    print(f"类别 {class_name}:")
    print(f" - 测试集数量: {total_count}")
    print(f" - 预测正确数量: {correct_count}")
    print(f" - 预测错误数量: {incorrect_count}")

### k-folds交叉验证

In [ ]:
from sklearn.datasets import load_iris
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import cross_val_predict, StratifiedKFold
from sklearn.metrics import classification_report

# 1. 加载鸢尾花数据集
data = load_iris()
X, y = data.data, data.target

# 2. 初始化 MLP 模型
mlp_1 = MLPClassifier(hidden_layer_sizes=(40,), max_iter=200, random_state=42)

# 3. 配置五折交叉验证
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# 4. 使用 cross_val_predict 进行交叉验证并获取预测
y_pred = cross_val_predict(mlp_1, X, y, cv=cv)

# 5. 输出分类报告
print("Classification Report:\n")
print(classification_report(y, y_pred, target_names=data.target_names))

mlp_2 = MLPClassifier(hidden_layer_sizes=(40,), max_iter=200, random_state=42)
# 数据集分割为训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
mlp_2.fit(X_train, y_train)

# 预测
y_pred = mlp_2.predict(X_test)

# 输出分类报告
print(classification_report(y_test, y_pred, target_names=data.target_names))

### Nos下训练MLP

In [ ]:
from scipy.stats import gmean
from utils.dataset_utils import get_classes_indexes_counts
from sklearn.metrics import confusion_matrix, roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report, accuracy_score
import scipy.io as sio  # 从.mat文件中读取数据集

mat_data = sio.loadmat('../data/dataset/Satellite.mat')
X = mat_data['X']  # 特征
y = mat_data['Y'][:, 0]  # 标签
classes, counts = get_classes_indexes_counts(y)
# 将数据集分为训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=100)

print(counts)
# 构建并训练MLP模型
mlp = MLPClassifier(hidden_layer_sizes=(20), max_iter=500, random_state=42)
mlp.fit(X_train, y_train)
y_proba = mlp.predict_proba(X_test)
# 预测和评估模型
y_pred = mlp.predict(X_test)
cm = confusion_matrix(y_test, y_pred)
# 计算 ROC AUC（ovo+macro）
auc_ovo_macro = roc_auc_score(y_test, y_proba, multi_class="ovo", average="macro")
# 计算每类召回率（每类正确预测个数 / 该类总数）
recall_per_class = cm.diagonal() / cm.sum(axis=1)
# 计算G-Mean
geometric_mean = gmean(recall_per_class)
# 计算准确率
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.2f}')
# 打印分类报告
print("Classification Report:")
print(classification_report(y_test, y_pred))
# 打印混淆矩阵
print("Confusion Matrix:")
print(cm)
print(f"最终的集成分类结果：Recall_Per_Class{recall_per_class}，Gmean：{geometric_mean}，mAUC：{auc_ovo_macro}")

### 平衡数据集后的训练

In [19]:
from scipy.stats import gmean
from utils.dataset_utils import get_classes_indexes_counts
from sklearn.metrics import confusion_matrix, roc_auc_score
from sklearn.model_selection import train_test_split, cross_val_predict, StratifiedKFold
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report, accuracy_score
from sklearn.preprocessing import StandardScaler
import scipy.io as sio  # 从.mat文件中读取数据集
import numpy as np

mat_data = sio.loadmat('../data/dataset/Nursery.mat')
X = mat_data['X']  # 特征
y = mat_data['Y'][:, 0]  # 标签
# 显示数据集分布
print("特征数据:", X.shape, "label:", y.shape)
# 统计每个类别的个数
classes, counts = get_classes_indexes_counts(y)  #np.argmax(y_onehot, axis=1)找最大值的索引，将0-1序列转化为0,1,2,3......的整数标签
print("每种类别的分布：", counts)
print("#########################划分数据集#########################")
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
# 数据标准化
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)
# 显示数据集分布
print("特征数据:", x_train.shape, "label:", y_train.shape)
# 统计每个类别的个数 
classes_train, counts_train = get_classes_indexes_counts(y_train)
# 计算每个类的权重
weights_train = (1 / counts_train.astype(float)) / np.sum(1 / counts_train.astype(float))
print("训练集每种类别的分布：", counts_train)
print("训练集每种类别的权重：", weights_train)
classes_test, counts_test = get_classes_indexes_counts(y_test)
print("测试集每种类别的分布：", counts_test)
print("#########################平衡数据集#########################")
# 确定每个类别的分布
num_instances = int(counts_train.min() * 1.0)  # 向下取整
num_instances_train = len(y_train)  # 取训练集的数量

print("最小数量:", num_instances)
# 在每个类别中随机的选择该数量的实例的索引
balanced_classes = np.array([])
for indexes in classes_train:
    random_selecte_indices = np.random.choice(indexes, size=num_instances, replace=False)
    balanced_classes = np.hstack((balanced_classes, random_selecte_indices))
balanced_classes = np.sort(balanced_classes).astype(int)
# 得到平衡的数据集
balanced_dataset_x = []
balanced_dataset_y = []
for index in balanced_classes:
    balanced_dataset_x.append(x_train[index])
    balanced_dataset_y.append(y_train[index])
balanced_dataset_x = np.array(balanced_dataset_x)
balanced_dataset_y = np.array(balanced_dataset_y).astype(int)
# 显示数据集分布
print("平衡的数据集的特征数据:", balanced_dataset_x.shape)
print("label:", balanced_dataset_y.shape)
# 统计每个类别的分布
classes_balanced_dataset, counts_balanced_dataset = get_classes_indexes_counts(balanced_dataset_y)
print("平衡的数据集中每种类别的分布：", counts_balanced_dataset)

# 构建并训练MLP模型
mlp = MLPClassifier(hidden_layer_sizes=(15), max_iter=200, random_state=42, learning_rate_init=0.1)
# mlp.fit(x_train, y_train)
mlp.fit(balanced_dataset_x, balanced_dataset_y)
y_proba = mlp.predict_proba(x_test)
# 预测和评估模型
y_pred = mlp.predict(x_test)
cm = confusion_matrix(y_test, y_pred)
# 计算 ROC AUC（ovo+macro）
auc_ovo_macro = roc_auc_score(y_test, y_proba, multi_class="ovo", average="macro")
# 计算每类召回率（每类正确预测个数 / 该类总数）
recall_per_class = cm.diagonal() / cm.sum(axis=1)
# 计算G-Mean
geometric_mean = gmean(recall_per_class)
# 计算准确率
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.2f}')
# 打印分类报告
print("Classification Report:")
print(classification_report(y_test, y_pred))
# 打印混淆矩阵
print("Confusion Matrix:")
print(cm)
print(f"最终的集成分类结果：Recall_Per_Class{recall_per_class}，Gmean：{geometric_mean}，mAUC：{auc_ovo_macro}")

######################计算三个目标值#########################
mlp_k_folds = MLPClassifier(hidden_layer_sizes=(15), max_iter=100, random_state=42, learning_rate_init=0.1)
# 3. 配置五折交叉验证
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# 4. 使用 cross_val_predict 进行交叉验证并获取预测
y_pred_k_folds = cross_val_predict(mlp_k_folds, x_train, y_train, cv=cv)
cm_k_folds = confusion_matrix(y_train, y_pred_k_folds)
tp_per_class = cm_k_folds.diagonal()  # 对角线元素表示每个类预测正确的个数，对角线求和，即所有预测正确的实例个数之和，计算Acc1
s_per_class = cm_k_folds.sum(axis=1)
Acc1 = np.sum(tp_per_class) / np.sum(s_per_class)  # Acc1
Acc2 = np.mean(tp_per_class.astype(float) / s_per_class.astype(float))  # Acc2
Acc3 = np.mean((tp_per_class.astype(float) / s_per_class.astype(float)) * weights_train)  # Acc3
# 输出Acc1, Acc2, Acc3
print("Acc1:", round(Acc1, 4))
print("Acc2:", round(Acc2, 4))
print("Acc3:", round(Acc3, 4))

特征数据: (12958, 8) label: (12958,)
每种类别的分布： [4320 4266 4044  328]
#########################划分数据集#########################
特征数据: (9070, 8) label: (9070,)
训练集每种类别的分布： [3041 2976 2821  232]
训练集每种类别的权重： [0.0616995  0.0630471  0.06651123 0.80874216]
测试集每种类别的分布： [1279 1290 1223   96]
#########################平衡数据集#########################
最小数量: 232
平衡的数据集的特征数据: (928, 8)
label: (928,)
平衡的数据集中每种类别的分布： [232 232 232 232]
Accuracy: 0.92
Classification Report:
              precision    recall  f1-score   support

           0       0.99      1.00      1.00      1279
           1       0.90      0.85      0.88      1290
           2       0.93      0.90      0.92      1223
           3       0.47      0.99      0.64        96

    accuracy                           0.92      3888
   macro avg       0.83      0.94      0.86      3888
weighted avg       0.93      0.92      0.92      3888

Confusion Matrix:
[[1279    0    0    0]
 [   6 1098   80  106]
 [   2  119 1102    0]
 [   0    1    0   95]]
最终的